In [1]:
import json
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

In [2]:
conn=http.client.HTTPSConnection("api.covid19api.com")
payload=""
headers={}
conn.request("GET","/summary",payload,headers)
res=conn.getresponse()
data=res.read().decode("UTF-8")

In [3]:
covid=json.loads(data)

In [4]:
df=pd.DataFrame(covid["Countries"])
df

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date,Premium
0,Afghanistan,AF,afghanistan,68,40141,3,1488,45,33561,2020-10-18T15:55:52Z,{}
1,Albania,AL,albania,273,16774,5,448,44,10001,2020-10-18T15:55:52Z,{}
2,Algeria,DZ,algeria,205,54203,5,1846,115,37971,2020-10-18T15:55:52Z,{}
3,Andorra,AD,andorra,0,3377,0,59,0,2057,2020-10-18T15:55:52Z,{}
4,Angola,AO,angola,240,7462,7,241,10,3022,2020-10-18T15:55:52Z,{}
...,...,...,...,...,...,...,...,...,...,...,...
184,Viet Nam,VN,vietnam,2,1126,0,35,0,1031,2020-10-18T15:55:52Z,{}
185,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-10-18T15:55:52Z,{}
186,Yemen,YE,yemen,0,2055,0,596,1,1336,2020-10-18T15:55:52Z,{}
187,Zambia,ZM,zambia,130,15789,0,346,28,14927,2020-10-18T15:55:52Z,{}


In [5]:
json_normalize(covid["Countries"],sep=",")

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
0,Afghanistan,AF,afghanistan,68,40141,3,1488,45,33561,2020-10-18T15:55:52Z
1,Albania,AL,albania,273,16774,5,448,44,10001,2020-10-18T15:55:52Z
2,Algeria,DZ,algeria,205,54203,5,1846,115,37971,2020-10-18T15:55:52Z
3,Andorra,AD,andorra,0,3377,0,59,0,2057,2020-10-18T15:55:52Z
4,Angola,AO,angola,240,7462,7,241,10,3022,2020-10-18T15:55:52Z
...,...,...,...,...,...,...,...,...,...,...
184,Viet Nam,VN,vietnam,2,1126,0,35,0,1031,2020-10-18T15:55:52Z
185,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-10-18T15:55:52Z
186,Yemen,YE,yemen,0,2055,0,596,1,1336,2020-10-18T15:55:52Z
187,Zambia,ZM,zambia,130,15789,0,346,28,14927,2020-10-18T15:55:52Z


In [6]:
covid=df.drop(columns=["CountryCode","Slug","Date","Premium"])

In [7]:
covid

,Country,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered
0,Afghanistan,68,40141,3,1488,45,33561
1,Albania,273,16774,5,448,44,10001
2,Algeria,205,54203,5,1846,115,37971
3,Andorra,0,3377,0,59,0,2057
4,Angola,240,7462,7,241,10,3022
...,...,...,...,...,...,...,...
184,Viet Nam,2,1126,0,35,0,1031
185,Western Sahara,0,10,0,1,0,8
186,Yemen,0,2055,0,596,1,1336
187,Zambia,130,15789,0,346,28,14927


In [8]:
m=folium.Map(tiles="stamen Terrain",min_zoom=1.5)
m

In [9]:
url="https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
country_shapes=f'{url}/world-countries.json'

In [10]:
folium.Choropleth(geo_data=country_shapes,min_zoom=2,name="COVID-19",data=covid,columns=["Country","TotalConfirmed"],key_on="feature.properties.name",fill_color="OrRd",nan_fill_color="black",legend_name="Confirmed Cases").add_to(m)
m

In [11]:
covid.update(covid["TotalConfirmed"].map("Total Confirmed:{}".format))
covid.update(covid["TotalRecovered"].map("Total Recovered:{}".format))

In [12]:
url="https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv"
cord=pd.read_csv(url)
cord.head()

,latitude,longitude,Country
0,33.939110,67.709953,Afghanistan
1,41.153332,20.168331,Albania
2,28.033886,1.659626,Algeria
3,-14.270972,-170.132217,American Samoa
4,42.546245,1.601554,Andorra


In [13]:
final=pd.merge(covid,cord,on="Country")

In [14]:
def plotit(f):
    folium.CircleMarker(location=[f.latitude,f.longitude],radius=5,weight=2,
                       popup=[f.Country,f.TotalConfirmed,f.TotalRecovered],
                        fill_color="#000000"
                       ).add_to(m)

In [15]:
final.apply(plotit,axis=1)
m.fit_bounds(m.get_bounds())
m

In [16]:
n=folium.Map(tiles="StamenToner",min_zoom=2)
n

In [17]:
death=final["TotalDeaths"].astype(float)

In [18]:
lat=final["latitude"].astype(float)

In [19]:
lon=final["longitude"].astype(float)

In [20]:
n.add_child(HeatMap(zip(lat,lon,death),radius=12))

In [21]:
def plotit(f):
    folium.CircleMarker(location=[f.latitude,f.longitude],radius=5,weight=2,popup=[f.Country,f.TotalDeaths],fill_color="#000000").add_to(n)
final.apply(plotit,axis=1)
n.fit_bounds(n.get_bounds())
n